In [ ]:
import huggingface_hub
hf_token = '' # put your User Access Tokens here
# ابتدا login کنید
huggingface_hub.login(token=hf_token)

# سپس وضعیت ورود را بررسی کنید
!hf auth whoami

/home/ai/Desktop/Fine-Tune/.env/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


user:  AM-Nateghi


In [2]:
# ================================================================
# Text Normalization
# ================================================================
ARABIC_TO_PERSIAN = {"\u064a": "ی", "\u0643": "ک", "\u06cc": "ی"}
PERSIAN_DIGITS_TO_EN = {f"۰۱۲۳۴۵۶۷۸۹"[i]: str(i) for i in range(10)}


def normalize_text(text: str) -> str:
    """Normalize Persian text"""
    if not isinstance(text, str):
        return ""
    for ar, fa in ARABIC_TO_PERSIAN.items():
        text = text.replace(ar, fa)
    for fa, en in PERSIAN_DIGITS_TO_EN.items():
        text = text.replace(fa, en)
    text = " ".join(text.split())
    text = text.replace(" ؟", "؟").replace("،", "، ")
    return " ".join(text.split())

In [3]:
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer
from peft import PeftModel

torch.cuda.empty_cache()

cptk = "google/gemma-3-270m"

# بارگذاری tokenizer
tokenizer = AutoTokenizer.from_pretrained("outputs/gemma3_qlora/phase1/checkpoint-500/")
# بارگذاری مدل پایه
base_model = AutoModelForCausalLM.from_pretrained(cptk)
# بارگذاری LoRA روی مدل پایه
model = PeftModel.from_pretrained(base_model, "outputs/gemma3_qlora/phase1/checkpoint-500/")
# انتقال به GPU
model.to("cuda")
print("ok!")

ok!


In [5]:
q_norm = normalize_text("چرا خواب های ما واقعی هستند؟ اصلا چطور میشه به وجودشون پی برد؟")
text = f"سوال: {q_norm}\nپاسخ: "
output = tokenizer.decode(
    model.generate(
        **tokenizer(text, return_tensors="pt").to(
            "cuda"
        ),
        max_new_tokens=512,
    )[0],
    skip_special_tokens=True
)
print(output)

سوال: چرا خواب های ما واقعی هستند؟ اصلا چطور میشه به وجودشون پی برد؟
پاسخ: 1. خواب ها رو هم به هم نزدیک میکنند، و هم به خودشان میگن "کتابات موجود در خواب" و همه خواب ها دارای بعضا یک کتاب است. 2. اما برای خواب های حقیقی و حقیقیون زندگی خواب انسان و برای خواب های حقیقی و حقیقیون فرار انسان ها در همین خواب ها در حقیقت عشق و نفرت و احساسات هستند و ما نمی دانیم حقیقت را از درون ما بگیریم. 3. انسان ها در خواب های واقعی و حقیقی زندگی میبینند که به خاطر ما انسان همون ما نیست، اما خواب های حقیقی و حقیقی هستهمن و ما رو بفهمیم که انسان ها در خواب های حقیقی زندگی و واقعا توحید نیستند.
شما به هر خواب های واقعی و حقیقی دنیا میگن با توجه به جنس ما هم فرق میکند.
پاسخ: 1. فرار از انسان ها بر روی خواب ها انسان هم برنده و فراتر از خواب انسان ها برنده نیستند و ما انسان ها برنده نیستیم و وقتی این موجودات دنیا براش رو ببرند هم در حقیقت فرار و لذت اندیشی و عشق ندارند و به اندازه ما و در حقیقت انسان های کاملا مردمی نیستند. 2. همون خواب ها انسان ها هم به فکر این عشق و هم به فکر این لذة است که میگن انسانها عاش